# E02 - Correciones

**Gonzalo Cano Padilla**

---

## Teoria

* Explica el modelo de regresión logística para clasificación. ¿Cómo se determina el umbral de decisión?

La regresión logística es un modelo que predice la probabilidad de que algo pertenezca a una clase, el umbral se determina eligiendo un punto a partir de cuando la probabilidad de considera positiva

* Explica la intuición de la máquina de soporte vectorial para clasificación. ¿Cómo se determina qué modelo es mejor? ¿Cuál es la mayor diferencia que tiene contra un modelo de regresión logística?

La máquina de soporte vectorial busca la frontera que mejor separa las clases, es decir la linea que maximiza la distancia entre la frontera y los puntos más cercanos (vectores de soporte).
Para saber que modelo es mejor se hace un cross-validation y se sacan métricas de score.
LA mayor diferencia es que SVM no modela probabilidades sino que busca la mejor frontera

* ¿Cuáles son los componentes principales en un MLP para clasificación? Dibuja un ejemplo y señaliza.

La capa de entrada -> capas ocultas -> capa de salida

* ¿Cuál es el procedimiento a seguir cuando los datos no son linealmente separables en una SVC?

Se usa el kernel trick para proyectar los datos a un espacio de mayor dimensión

* Describe qué es un hiperparámetro. ¿Por qué es importante ajustarlos? Da dos ejemplos de hiperparámetros.

Son valores que controlan el comportamiento del modelo, son ajustables mediante optimizaciones.
C penaliza los errores, gamma es la curvatura del kernel RBF

* Dibuja un diagrama de flujo para describir el proceso de optimización Bayesiana.

1. Función objetivo
2. Probar con valores iniciales de hiperparámetros
3. Construir un modelo probabilístico (gaussiano) que estima como se comporta la función objetivo
4. Elegir el siguiente punto a probar
5. Repetir hasta encontrar los hiperparámetros optimos

* ¿Qué es la curva ROC y cómo se usa para evaluar el desempeño de un modelo?

Muestra cómo cambia el desempeño del modelo al mover el umbral de decisión, eje X: tasa de falsos positivos, eje Y: tasa de verdaderos positivos

* Describe un espacio de Hilbert.

Es un espacio donde puedes para hacer operaciones si los datos tienen infinitas dimensiones.

* ¿Qué significa que una función de costo sea convexa? ¿Qué beneficios hay de que un modelo tenga una función de costo convexa?

Significa que el algoritmo de optimización siempre encontrara el mejor punto, si no es convexa pueden existir varios mínimos locales y el modelo podriía quedarse en uno malo

* Piensa en los 3 modelos aprendidos en este parcial: ¿En qué situaciones usarías cada uno y por qué?
- La regresión logística cuando los datos son lineales y quieres interpretar las probabilidades
- SVM cuando los datos no son lineales y quieres una frontera bien definida
- MLP cuando los datos son complejos, no lineales y hay muchas variables o interacciones

---

## Parte práctica

In [1]:
# Ejercicio 1
import numpy as np
from skopt import gp_minimize
from skopt.space import Real

def f(params):
    x, y, z = params
    parte_1 = (6*x - 2)**2 * np.sin(12*x - 4)
    parte_2 = (6*y - 2)**2 * np.cos(12*y - 4)
    parte_3 = (6*z - 2)**2 * np.sin(12*z - 4)
    return parte_1 + parte_2 + parte_3

espacio_busqueda = [
    Real(0,1, name='x'),
    Real(0,1, name='y'),
    Real(0,1, name='z'),
]

res = gp_minimize(
    func=f,
    dimensions=espacio_busqueda,
    n_calls=15,
    n_initial_points=5,
    acq_func='EI',
    random_state=42
)

print('mínimo encontrado:', res.fun)
print('x, y, z:', res.x)

mínimo encontrado: -13.284416950202857
x, y, z: [0.7015747936280105, 0.0, 0.7638670714529164]


En este caso es mejor usar optimización bayesiana porque tenemos un espacio de busqueda continuo y no convexo y es más facil de evaluar

In [13]:
# Ejercicio 2
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

data = pd.read_csv('adidas.csv')
data = data.dropna()

y_raw = data['average_rating']
y = y_raw>=4.3
X = data.drop(columns=['average_rating'])

num_features = ['selling_price','reviews_count']
cat_features = ['url', 'name', 'sku', 'original_price', 'currency',
       'availability', 'color', 'category', 'source', 'source_website',
       'breadcrumbs', 'description', 'brand', 'images', 'country', 'language', 'crawled_at']

numerical_transformer = StandardScaler()
preprocess = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features),
        ('num', numerical_transformer, num_features)
    ]
)

lr=LogisticRegression()
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=42)

pipeline = Pipeline([
    ('preprocess', preprocess),
    ('classifier', lr),
])


auc = make_scorer(roc_auc_score, needs_proba=True, multi_class='ovr')

pipeline.fit(Xtrain, ytrain)
auc = pipeline.score(Xtest, ytest, scoring='roc_auc')
auc

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


0.9375

el modelo tiene aproximadamente un 93.75 % de probabilidad de asignar una mayor puntuación de producto bueno al producto correcto.

In [24]:
from scipy.stats import norm

# Matriz X transformada
Xtrain_t = pipeline.named_steps['preprocess'].transform(Xtrain)
Xtrain_t = Xtrain_t.toarray()

# Coeficientes e intercepto
b0 = pipeline.named_steps['classifier'].intercept_[0]
b = pipeline.named_steps['classifier'].coef_.flatten()

# Predicciones
pred = 1 / (1 + np.exp(-(b0 + Xtrain_t @ b)))

# Error p(1-p) y matriz diagonal
error = pred * (1 - pred)
V = np.diagflat(error)

# Matriz de covarianza
n = len(Xtrain_t)
X_design = np.hstack((np.ones((n, 1)), Xtrain_t))
cov = np.linalg.pinv(X_design.T @ V @ X_design)

# Error estándar
se = np.sqrt(np.diag(cov))

# Estadísticos z
beta = np.r_[b0, b]
z = beta / se

# P-values
p_values = 2 * (1 - norm.cdf(np.abs(z)))

# Resultados
print(f"Intercepto (b0): {b0}")
print(f"Coeficientes (betas): {b}")
print(f"P-values:{p_values}")

Intercepto (b0): 3.203213309487905
Coeficientes (betas): [ 0.01416769  0.0137258  -0.39767378 ...  0.12141319 -0.43158245
  1.03445432]
P-values:
[0.99999608        nan        nan ...        nan        nan 0.99676999]


/var/folders/20/xvvxf3fd0bsc8mhjgvpylzzc0000gn/T/ipykernel_20443/1298864727.py:24: RuntimeWarning: invalid value encountered in sqrt
  se = np.sqrt(np.diag(cov))


In [27]:
# Crear DataFrame con betas y p-values
tabla = pd.DataFrame({
    'Beta': np.r_[b0, b],
    'P_value': p_values
})
tabla

,Beta,P_value
0,3.203213,0.999996
1,0.014168,NaN
2,0.013726,NaN
3,-0.397674,NaN
4,0.008496,0.999999
...,...,...
3360,0.013121,0.999896
3361,0.018698,0.999951
3362,0.121413,NaN
3363,-0.431582,NaN
